GPS Data 간단히 하기

In [2]:
import pandas as pd
from rdp import rdp
import folium
from matplotlib import pyplot as plt

In [3]:
#gps data 불러오기
gps_data = pd.read_csv('Task1_20191105.csv')

In [4]:
gps_data.head()

,003_vmcu accel pedal depth,003_vmcu accel pedal related,003_vmcu brake lamp,003_vmcu brake related,003_vmcu brakes on,003_vmcu d,003_vmcu n,003_vmcu p,003_vmcu r,003_vmcu real vehicle speed,...,004_calc average cell voltage,created_on,MacAddress,before_parsing,latitude,longitude,gps_speed,ev_speed_kmh,accuracy,rownum
0,0.0%,0.0,0.0,90.0,0.0,0.0,0.0,0.0,1.0,4.16mph,...,4.120833333333334V,2019-11-05 15:03:07,00:1D:A5:00:1B:86,[B@ed30c1e,0.0,0.0,0.0,6.6976,0.0,1
1,0.0%,0.0,0.0,90.0,0.0,0.0,0.0,0.0,1.0,3.86mph,...,4.120833333333334V,2019-11-05 15:03:09,00:1D:A5:00:1B:86,[B@df9a7ec,0.0,0.0,0.0,6.2146,0.0,2
2,0.0%,0.0,0.0,90.0,0.0,0.0,0.0,0.0,1.0,4.52mph,...,4.119791666666667V,2019-11-05 15:03:13,00:1D:A5:00:1B:86,[B@d8cb6f,0.0,0.0,0.0,7.2772,0.0,3
3,0.0%,0.0,1.0,89.0,1.0,0.0,0.0,0.0,1.0,0.0mph,...,4.120833333333334V,2019-11-05 15:03:17,00:1D:A5:00:1B:86,[B@77cde55,0.0,0.0,0.0,0.0000,0.0,4
4,0.0%,0.0,0.0,90.0,0.0,0.0,0.0,0.0,1.0,4.16mph,...,4.119791666666667V,2019-11-05 15:03:23,00:1D:A5:00:1B:86,[B@e8bc10f,0.0,0.0,0.0,6.6976,0.0,5


In [5]:
#gps데이터에서 생성시간, 위도, 경도 데이터만 나타내기
gps_data = gps_data[['003_vmcu real vehicle speed','created_on', 'latitude', 'longitude']]
gps_data.rename(columns = {'003_vmcu real vehicle speed' : 'speed'}, inplace = True)

In [6]:
#이상치 제거 - 위도, 경도가 0이고, 속도가 0인 것(정지상태)
gps_data = gps_data[gps_data.latitude !=0]
gps_data = gps_data[gps_data.longitude !=0]
gps_data = gps_data[gps_data.speed !="0.0mph"]

#생성시각을 index로 지정
gps_data.set_index('created_on', inplace = True)
gps_data.head()

,speed,latitude,longitude
created_on,,,
2019-11-05 15:05:40,0.94mph,37.492376,127.039125
2019-11-05 15:05:43,1.25mph,37.492412,127.039117
2019-11-05 15:05:46,1.38mph,37.492444,127.039131
2019-11-05 15:05:49,1.17mph,37.492452,127.039144
2019-11-05 15:05:51,1.83mph,37.492454,127.039149


In [7]:
gps_data.shape

(4333, 3)

In [8]:
#Florian Wilhelm - Handling GPS Data with Python https://youtu.be/9Q8nEA_0ccg
simple_coords = rdp(gps_data[['longitude','latitude']].values, epsilon=1e-5)

In [9]:
simple_coords.shape

(1921, 2)

In [10]:
#simple_coords를 array type에서 dataframe 형식으로 바꾸기
gps_simple_data = pd.DataFrame({'longitude':simple_coords[:,0]
                              , 'latitude': simple_coords[:,1]})

In [11]:
gps_simple_data.head()

,longitude,latitude
0,127.039125,37.492376
1,127.039117,37.492412
2,127.039155,37.492456
3,127.039162,37.492369
4,127.039227,37.492283


In [12]:
def find_slope(a,b):
    slope = (gps_simple_data.loc[a,'longitude']-gps_simple_data.loc[b,'longitude'])/((gps_simple_data.loc[a,'latitude']-gps_simple_data.loc[b,'latitude']))
    return slope

delete = 0

len = gps_simple_data.shape[0]
for i in range(1,len-1):
    if delete == 0:
        before = gps_simple_data.loc[i,'longitude'] - gps_simple_data.loc[i-1,'longitude']
        after = gps_simple_data.loc[i+1,'longitude'] - gps_simple_data.loc[i, 'longitude']
        if before*after < 0:
            gps_simple_data = gps_simple_data.drop(i+1,0)
            delete = 2
    else:
        delete = delete-1
    

In [13]:
gps_simple_data.shape

(1878, 2)

In [14]:
gps_simple_data

,longitude,latitude
0,127.039125,37.492376
1,127.039117,37.492412
3,127.039162,37.492369
4,127.039227,37.492283
6,127.039251,37.492094
7,127.039225,37.492075
9,127.040427,37.492156
10,127.040797,37.492285
11,127.040969,37.492317
12,127.042102,37.492636


Mplleaflet으로 나타내기

In [15]:
import mplleaflet

In [23]:
fig = gps_simple_data.plot(gps_simple_data."longitude", gps_simple_data."latitude")
mplleaflet.display(fig=fig) # show(fig=fig) to display in new window

SyntaxError: invalid syntax (<ipython-input-23-ed38d2223afb>, line 1)

Folium으로 나타내기

In [25]:
#base_map 형식 지정하기
def generateBaseMap(default_location = [37.492397, 127.039084], default_zoom_start = 12):
    base_map = folium.Map(location = default_location, control_scale = True, zoom_start = default_zoom_start)
    return base_map

#base map 그리기
base_map = generateBaseMap()

#rdp한 simple data로 그리기
for coord in gps_simple_data[['latitude','longitude']].values:
    folium.CircleMarker(location=[coord[0], coord[1]], radius=1, color='red').add_to(base_map)

folium.PolyLine(gps_simple_data[['latitude','longitude']].values).add_to(base_map)

base_map.save("gps_rdp_simple.html")